In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [3]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.2.4
2.5


In [4]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [5]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [6]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [7]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [8]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x000001B85C6B7EE0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x000001B85C6B7EE0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x000001B85C6B7EE0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0

In [9]:
#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [05:38<00:00,  1.69s/it]
[MainProcess/INFO] experiments finished


In [10]:
#observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,95,333.476201,1.0,0.708616,283.919213,10,0.703246,139.613565,10,0.164605,...,8,4,2,1,9,7,4,4,0,dikesnet
1,13,56.171626,10,0.291520,241.904517,1.0,0.433867,291.949465,1.5,0.974558,...,8,4,2,1,9,7,4,5,0,dikesnet
2,102,183.095304,1.0,0.435061,221.743455,10,0.867275,30.809662,1.5,0.272286,...,8,4,2,1,9,7,4,6,0,dikesnet
3,128,217.332123,1.5,0.940079,50.867157,1.0,0.345007,299.134335,1.0,0.466083,...,8,4,2,1,9,7,4,7,0,dikesnet
4,79,137.855682,1.0,0.817086,231.264540,1.5,0.175335,178.583609,10,0.456066,...,8,4,2,1,9,7,4,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,121,340.017164,1.0,0.188098,311.866439,1.0,0.750913,227.920928,1.0,0.232165,...,2,2,8,8,1,3,3,49,3,dikesnet
196,0,161.823478,1.5,0.600885,118.160528,1.5,0.898120,188.361301,1.0,0.556777,...,2,2,8,8,1,3,3,50,3,dikesnet
197,109,199.567292,10,0.960406,211.943986,1.0,0.155641,93.912235,10,0.004541,...,2,2,8,8,1,3,3,51,3,dikesnet
198,12,152.462231,10,0.531759,179.912228,1.5,0.247109,317.107356,1.5,0.141249,...,2,2,8,8,1,3,3,52,3,dikesnet


In [99]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{' 4_RfR 0':1,
                                  '4_RfR 1':0,
                                  '4_RfR 2':0,
                                  'A.3_DikeIncreas e 0':0})]#,
           #Policy('policy 2', **{'4_RfR 0':1,
            #                      '4_RfR 1':1,
             #                     '4_RfR 2':1,
              #                    'A.5_DikeIncrease 0':5}),
           #Policy('policy 3', **{'1_RfR 0':1,
            #                      '2_RfR 1':1,
             #                     '3_RfR 2':1,
              #                    'A.3_DikeIncrease 0':5})]

In [100]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)

#pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

results



[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 1 policies * 1 model(s) = 100 experiments
100%|████████████████████████████████████████| 100/100 [01:21<00:00,  1.22it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


(    A.0_ID flood wave shape    A.1_Bmax A.1_Brate  A.1_pfail    A.2_Bmax  \
 0                        38  157.434381        10   0.582899   36.063275   
 1                         6   90.999441        10   0.181257  185.083316   
 2                        83  310.689111       1.0   0.777645  313.423851   
 3                       100   97.767512        10   0.394908  120.691586   
 4                        90  172.616629       1.5   0.268788  326.051549   
 ..                      ...         ...       ...        ...         ...   
 95                       92  333.481048       1.5   0.379621  152.778146   
 96                       35  273.899015       1.5   0.653435  169.555468   
 97                       17  105.331055       1.0   0.924291  259.567751   
 98                      124   45.810590       1.5   0.447969  114.161594   
 99                       72  114.081923        10   0.517248  100.746123   
 
    A.2_Brate  A.2_pfail    A.3_Bmax A.3_Brate  A.3_pfail  ...  \
 0      

In [101]:
import pandas as pd
a,b = results
df = pd.DataFrame(b)
df

#a = np.min(df['A.3 Total Costs'])#



,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,9.744930e+07,0.000641,8.803888e+07,0.002444,1.572138e+08,0.032536,1.742061e+07,0.000000,1.398593e+08,0.008237,256100000.0,13016.378190
1,2.857594e+09,0.218374,8.997764e+07,0.002172,5.279261e+07,0.003019,3.002341e+07,0.000658,2.095293e+08,0.014654,256100000.0,42600.997001
2,9.246005e+07,0.000000,8.526650e+07,0.001920,1.066867e+08,0.015781,1.257243e+08,0.006188,7.221842e+07,0.000000,256100000.0,13406.401432
3,2.741947e+08,0.012916,8.225555e+07,0.001607,4.141836e+07,0.000364,2.004591e+07,0.000214,7.221842e+07,0.000000,256100000.0,3142.093881
4,1.577481e+09,0.101550,1.556983e+08,0.009962,4.030700e+07,0.000000,1.742061e+07,0.000000,7.221842e+07,0.000000,256100000.0,19235.506573
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.316110e+09,0.099663,1.006199e+09,0.094694,3.716000e+07,0.000000,1.623000e+07,0.000000,6.813000e+07,0.000000,256100000.0,67902.171822
96,1.523259e+08,0.006479,8.668131e+07,0.002440,1.622950e+08,0.030355,2.391575e+07,0.000479,4.816058e+08,0.049588,256100000.0,39545.694831
97,8.571000e+07,0.000000,1.251574e+08,0.006148,8.403476e+07,0.009505,1.611109e+08,0.006625,6.813000e+07,0.000000,256100000.0,15634.060695
98,6.969892e+08,0.040901,7.134179e+07,0.000299,1.432688e+09,0.231718,1.623000e+07,0.000000,8.075956e+08,0.061105,256100000.0,101500.221876


In [102]:
print(np.min(df['A.3 Total Costs']))
print(np.average(df['A.3 Total Costs']))
print(np.max(df['A.3 Total Costs']))

37160000.00000301
415346319.41290045
2142267118.3842742


In [103]:
print(np.min(df['A.3_Expected Number of Deaths']))
print(np.average(df['A.3_Expected Number of Deaths']))
print(np.max(df['A.3_Expected Number of Deaths']))

0.0
0.08017002562217167
0.4092784227625138


In [104]:
print(np.min(df['RfR Total Costs']))
print(np.average(df['RfR Total Costs']))
print(np.max(df['RfR Total Costs']))

256100000.00000003
256100000.00000003
256100000.00000003


In [105]:
print(np.min(df['Expected Evacuation Costs']))
print(np.average(df['Expected Evacuation Costs']))
print(np.max(df['Expected Evacuation Costs']))

3142.0938808781575
44874.87715596213
133733.3544270394
